In [1]:
!pip install transformer_lens
!pip install einops
!pip install fancy_einsum
!pip install jaxtyping
!pip install circuitsvis
!pip install -U kaleido

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import os; os.environ['ACCELERATE_DISABLE_RICH'] = "1"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from IPython.display import HTML, IFrame
from functools import partial
import sys
import einops
from fancy_einsum import einsum
import torch.nn.functional as F
from dataclasses import dataclass
from transformer_lens import HookedTransformer
from transformer_lens.utils import gelu_new, tokenize_and_concatenate
from transformer_lens import patching
import torch as t
from torch import Tensor
import torch.nn as nn
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import re
import itertools
import math
from tqdm import tqdm
from typing import List, Optional, Callable, Tuple, Dict, Literal, Set, Union
from jaxtyping import Float, Int
from transformers.models.gpt2.tokenization_gpt2_fast import GPT2TokenizerFast
from collections import defaultdict
from rich.table import Table, Column
from rich import print as rprint
import datasets
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
import webbrowser
#import gdown
import circuitsvis as cv
from itertools import combinations, permutations

#from circuitsvis.attention import attention_heads

from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformer_lens.utils import to_numpy

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

t.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")
device

device(type='cpu')

### Model Import

In [4]:
# import model
small = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device = device
)

Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
# import model
medium = HookedTransformer.from_pretrained(
    "gpt2-medium",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device = device
)

In [11]:
# import model
large = HookedTransformer.from_pretrained(
    "gpt2-large",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device = device
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-large into HookedTransformer


In [12]:
# import model
xl = HookedTransformer.from_pretrained(
    "gpt2-xl",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device = device
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-xl into HookedTransformer


In [13]:
import sys
sys.path.append('..')
import importlib
import deductive_dataset
from deductive_dataset import DeductiveDataset
importlib.reload(deductive_dataset)
print('Imported')

Imported


In [14]:
def logits_to_ave_logit_diff(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch 2"],
    per_prompt: bool = False
) -> Union[Float[Tensor, ""], Float[Tensor, "batch"]]:
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''
    # SOLUTION
    # Only the final logits are relevant for the answer
    final_logits: Float[Tensor, "batch d_vocab"] = logits[:, -1, :]
    # Get the logits corresponding to the indirect object / subject tokens respectively
    answer_logits: Float[Tensor, "batch 2"] = final_logits.gather(dim=-1, index=answer_tokens)
    # Find logit difference
    correct_logits, incorrect_logits = answer_logits.unbind(dim=-1)
    answer_logit_diff = correct_logits - incorrect_logits
    #print(correct_logits, incorrect_logits, answer_logit_diff)
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

### Dataset import

In [15]:
N = 15
seed = 317
# 0
s_dataset = DeductiveDataset(
            N=N, # number of pairs (which omposed of 6)
            seed=seed,
            device=device,
            type='symbolic'
        )


(' O', ' A', ' W')
(' O', ' W', ' A')
(' A', ' O', ' W')
(' A', ' W', ' O')
(' W', ' O', ' A')
(' W', ' A', ' O')
(' Y', ' M', ' A')
(' Y', ' A', ' M')
(' M', ' Y', ' A')
(' M', ' A', ' Y')
(' A', ' Y', ' M')
(' A', ' M', ' Y')
(' G', ' H', ' R')
(' G', ' R', ' H')
(' H', ' G', ' R')
(' H', ' R', ' G')
(' R', ' G', ' H')
(' R', ' H', ' G')
(' O', ' W', ' P')
(' O', ' P', ' W')
(' W', ' O', ' P')
(' W', ' P', ' O')
(' P', ' O', ' W')
(' P', ' W', ' O')
(' O', ' U', ' J')
(' O', ' J', ' U')
(' U', ' O', ' J')
(' U', ' J', ' O')
(' J', ' O', ' U')
(' J', ' U', ' O')
(' N', ' P', ' L')
(' N', ' L', ' P')
(' P', ' N', ' L')
(' P', ' L', ' N')
(' L', ' N', ' P')
(' L', ' P', ' N')
(' U', ' D', ' C')
(' U', ' C', ' D')
(' D', ' U', ' C')
(' D', ' C', ' U')
(' C', ' U', ' D')
(' C', ' D', ' U')
(' Q', ' N', ' W')
(' Q', ' W', ' N')
(' N', ' Q', ' W')
(' N', ' W', ' Q')
(' W', ' Q', ' N')
(' W', ' N', ' Q')
(' L', ' C', ' J')
(' L', ' J', ' C')
(' C', ' L', ' J')
(' C', ' J', ' L')
(' J', ' L',

In [29]:
import sys
sys.path.append('..')
import importlib
import deductive_dataset
from deductive_dataset import DeductiveDataset
importlib.reload(deductive_dataset)
print('Imported')

N = 90
seed = 317
# 0
ni_dataset = DeductiveDataset(
             tokenizer = None,
            N=N, # number of pairs (which omposed of 6)
            seed=seed,
            device=device,
            type='inconsistent',
            template_type = 'CAT',
            saved_file = True
        )


Imported


In [19]:
import sys
sys.path.append('..')
import importlib
import deductive_dataset
from deductive_dataset import DeductiveDataset
importlib.reload(deductive_dataset)
print('Imported')

N = 90
seed = 317
# 0
nc_dataset = DeductiveDataset(
            tokenizer = None,
            N=N, # number of pairs (which omposed of 6)
            seed=seed,
            device=device,
            type='consistent',
            template_type = 'CAT',
            saved_file = True
        )


Imported


In [ ]:
for a, b in zip(ni_dataset.sentences, ni_dataset.labels):
    print(a, b)

In [20]:
for a, b in zip(nc_dataset.sentences, nc_dataset.labels):
    print(a, b)

All men are humans. All humans are mortal. Therefore, all men are  mortal
All dens are units. All units are parts. Therefore, all dens are  parts
All pets are animals. All animals are organisms. Therefore, all pets are  organisms
All openings are starts. All starts are beginnings. Therefore, all openings are  beginnings
All fronts are sides. All sides are surfaces. Therefore, all fronts are  surfaces
All fences are boundaries. All boundaries are lines. Therefore, all fences are  lines
All monitors are displays. All displays are demonstrations. Therefore, all monitors are  demonstrations
All necks are parts. All parts are objects. Therefore, all necks are  objects
All governors are executives. All executives are administrators. Therefore, all governors are  administrators
All explosions are increases. All increases are amounts. Therefore, all explosions are  amounts
All compositions are creations. All creations are artifacts. Therefore, all compositions are  artifacts
All taxis are cars

In [ ]:
a= small.to_str_tokens(nc_dataset.B)
for i in a:
    if len(i) != 2:
        print(i)
        break

In [ ]:
nc_dataset.sentences[0], nc_dataset.labels[0]

('All dens are units. All units are parts. Therefore, all dens are', ' parts')

In [ ]:
s_dataset.sentences[0]

'All O are A. All A are W. Therefore, all O are'

In [ ]:
len(model.to_str_tokens(s_dataset.sentences[0], prepend_bos=False))

NameError: name 'model' is not defined

### GPT data sampling function

In [21]:
class TransformerSampler:

    def __init__(self, model, tokenizer):
        self.model = model
        self.cfg = model.cfg
        self.tokenizer = tokenizer

    @t.inference_mode()
    def sample(self, prompt, max_tokens_generated=100, verbose=False, **kwargs):
        # SOLUTION
        self.model.eval()
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(device)[0]

        for i in range(max_tokens_generated):
            # Get new logits (make sure we don't pass in more tokens than the model's context length)
            logits = self.model(input_ids[None, -self.cfg.n_ctx:])
            # We only take logits for the last token, because this is what we're sampling
            logits = logits[0, -1]
            # Get next token (as a tensor of size (1, 1) so we can concat it to input_ids)
            next_token = t.tensor([TransformerSampler.sample_next_token(input_ids, logits, **kwargs)], device=device)
            #print(f"=>{self.model.to_string(next_token)}")
            # Create new input ids string, with shape (1, old_seq_len + 1)

            input_ids = t.cat([input_ids, next_token], dim=-1)
            # Print out results, if required
            if verbose:
                print(self.tokenizer.decode(input_ids), end="\r")
            # If our new token was the end-of-text token, stop
            if next_token == getattr(self.tokenizer, "eos_token_id", None):
                break

        return self.tokenizer.decode(input_ids)

    @t.inference_mode()
    def beam_search(
        self,
        prompt: str,
        num_return_sequences: int,
        num_beams: int,
        max_new_tokens: int,
        no_repeat_ngram_size: int = 0,
        verbose=False
    ) -> List[Tuple[float, t.Tensor]]:
        '''
        Returns a string of autoregressively generated text, starting from the prompt.

        Sampling terminates at max_tokens_generated, or when the model generates an
        end-of-sequence token.

        kwargs are passed to sample_next_token, to give detailed instructions on how
        new tokens are chosen.
        '''
        pass


    @staticmethod
    def sample_next_token(
        input_ids: Int[Tensor, "seq_len"],
        logits: Float[Tensor, "d_vocab"],
        temperature=1.0,
        top_k=0,
        top_p=0.0,
        frequency_penalty=0.0,
        seed=None
    ):
        assert input_ids.ndim == 1, "input_ids should be a 1D sequence of token ids"
        assert temperature >= 0, "Temperature should be non-negative"
        assert 0 <= top_p <= 1.0, "Top-p must be a probability"
        assert 0 <= top_k, "Top-k must be non-negative"
        assert not (top_p != 0 and top_k != 0), "At most one of top-p and top-k supported"

        # Set random seeds for reproducibility
        if seed is not None:
            t.manual_seed(seed)
            np.random.seed(seed)

        # Apply all the specialized sampling methods
        if temperature == 0:
            return TransformerSampler.greedy_search(logits)
        elif temperature != 1.0:
            logits = TransformerSampler.apply_temperature(logits, temperature)
        if frequency_penalty != 0.0:
            logits = TransformerSampler.apply_frequency_penalty(input_ids, logits, frequency_penalty)
        if top_k > 0:
            return TransformerSampler.sample_top_k(logits, top_k)
        if top_p > 0.0:
            return TransformerSampler.sample_top_p(logits, top_p)
        return TransformerSampler.sample_basic(logits)


    @staticmethod
    def greedy_search(logits: Float[Tensor, "d_vocab"]) -> int:
        '''
        Returns the most likely token (as an int).
        '''
        out = logits.argmax().item()
        return out


    @staticmethod
    def apply_temperature(logits: Float[Tensor, "d_vocab"], temperature: float) -> Float[Tensor, "d_vocab"]:
        assert temperature > 0
        return logits / temperature

    @staticmethod
    def apply_frequency_penalty(input_ids: Int[Tensor, "seq_len"], logits: Float[Tensor, "d_vocab"], freq_penalty: float) -> Float[Tensor, "d_vocab"]:
        '''
        Applies a frequency penalty to the logits.
        '''
        pass

    @staticmethod
    def sample_basic(logits: t.Tensor) -> int:
        '''
        logits: shape (vocab_size, ) - unnormalized log-probabilities

        Return: a sampled token
        '''
        distribution = t.distributions.categorical.Categorical(logits=logits)
        out = distribution.sample().item()
        assert isinstance(out, int)
        return out

    @staticmethod
    def sample_top_k(logits: t.Tensor, top_k: int) -> int:
        '''
        logits: shape (vocab_size, ) - unnormalized log-probabilities
        top_k: only consider this many of the most likely tokens for sampling

        Return: a sampled token
        '''
        top_logits, top_idx = t.topk(logits, top_k)
        idx = t.distributions.categorical.Categorical(logits=top_logits).sample()
        return top_idx[idx].item()

    @staticmethod
    def sample_top_p(logits: Float[Tensor, "d_vocab"], top_p: float, min_tokens_to_keep: int = 1) -> int:
        '''
        Samples from the most likely tokens which make up at least p cumulative probability.
        '''
        pass

In [22]:
sampler = TransformerSampler(medium, medium.tokenizer)

prompt = "Jingle bells, jingle bells, jingle all the way"
print(f"Greedy decoding with prompt: {prompt!r}\n")

output = sampler.sample(prompt, max_tokens_generated=8, temperature=0.0)
print(f"Your model said: {output!r}\n")

expected = "Jingle bells, jingle bells, jingle all the way up to the top of the mountain."

Greedy decoding with prompt: 'Jingle bells, jingle bells, jingle all the way'

Your model said: 'Jingle bells, jingle bells, jingle all the way to the end of the world.\n'



In [23]:
logits = t.tensor([1, 2]).log()

cold_logits = TransformerSampler.apply_temperature(logits, temperature=0.001)
print('A low temperature "sharpens" or "peaks" the distribution: ', cold_logits)
t.testing.assert_close(cold_logits, 1000.0 * logits)

hot_logits = TransformerSampler.apply_temperature(logits, temperature=1000.0)
print("A high temperature flattens the distribution: ", hot_logits)
t.testing.assert_close(hot_logits, 0.001 * logits)

print("Tests passed!")

A low temperature "sharpens" or "peaks" the distribution:  tensor([  0.0000, 693.1472])
A high temperature flattens the distribution:  tensor([0.0000, 0.0007])
Tests passed!


In [30]:
sampler = TransformerSampler(model, model.tokenizer)

your_prompt = "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."
output = sampler.sample(your_prompt, temperature=0.7, top_k=40, max_tokens_generated=5)
rprint(f"Your model said:\n\n[bold dark_orange]{output}")

NameError: name 'model' is not defined

In [ ]:
prompt = s_dataset.sentences[0]
output = sampler.sample(prompt, temperature=0, top_k=40, max_tokens_generated=1)
rprint(f"Your model said:\n\n[bold dark_orange]{output}")

=> W


Your model said:

All O are A. All A are W. Therefore, all O are W

# Accuracy Test

In [26]:
medium_sampler = TransformerSampler(medium, medium.tokenizer)

In [31]:
def get_acc(prompts, labels, model, sampler, temperature=0.0, top_k=0, max_tokens_generated=1):

    def check(prompt, answer):
        logits, cache = model.run_with_cache(prompt, device=device, prepend_bos=False)
        output = model.tokenizer.decode(logits[:,-1].argmax(dim=-1))
        if answer == output:
            #print(prompt, output, answer, "right")
            return 1
        else:
            #print(prompt, output, answer, "wrong")
            return 0

    acc = 0
    for prompt, label in zip(prompts, labels):

        output = sampler.sample(prompt, temperature=0, top_k=40, max_tokens_generated=1)
        output_last = model.to_str_tokens(output, prepend_bos=False)[-1]
        #print(label, output_last)
        if label == output_last:
            acc += 1

    return acc/len(prompts)

### Small

In [32]:
small_sampler = TransformerSampler(small, small.tokenizer)

In [33]:
acc_s = [get_acc(s_dataset.sentences, s_dataset.labels, small, small_sampler)]
acc_nc = [get_acc(nc_dataset.sentences, nc_dataset.labels, small, small_sampler)]
acc_ni = [get_acc(ni_dataset.sentences, ni_dataset.labels, small, small_sampler)]

In [34]:
acc_s, acc_nc, acc_ni

([0.05555555555555555], [0.2111111111111111], [0.29213483146067415])

### Medium

In [35]:
medium_sampler = TransformerSampler(medium, medium.tokenizer)

In [36]:
nc_dataset.sentences[0], nc_dataset.labels[0]

('All men are humans. All humans are mortal. Therefore, all men are',
 ' mortal')

In [37]:
acc_s.append(get_acc(s_dataset.sentences, s_dataset.labels, medium, medium_sampler))
acc_nc.append(get_acc(nc_dataset.sentences, nc_dataset.labels, medium, medium_sampler))
acc_ni.append(get_acc(ni_dataset.sentences, ni_dataset.labels, medium, medium_sampler))

In [38]:
acc_s, acc_nc, acc_ni

([0.05555555555555555, 0.6666666666666666],
 [0.2111111111111111, 0.4666666666666667],
 [0.29213483146067415, 0.5730337078651685])

## Large

In [39]:
large_sampler = TransformerSampler(large, large.tokenizer)

acc_s.append(get_acc(s_dataset.sentences, s_dataset.labels, large, large_sampler))
acc_nc.append(get_acc(nc_dataset.sentences, nc_dataset.labels, large, large_sampler))
acc_ni.append(get_acc(ni_dataset.sentences, ni_dataset.labels, large, large_sampler))

In [40]:
acc_s, acc_nc, acc_ni

([0.05555555555555555, 0.6666666666666666, 0.9222222222222223],
 [0.2111111111111111, 0.4666666666666667, 0.4888888888888889],
 [0.29213483146067415, 0.5730337078651685, 0.6629213483146067])

### XL

In [41]:
xl_sampler = TransformerSampler(xl, xl.tokenizer)

acc_s.append(get_acc(s_dataset.sentences, s_dataset.labels, xl, xl_sampler))
acc_nc.append(get_acc(nc_dataset.sentences, nc_dataset.labels, xl, xl_sampler))
acc_ni.append(get_acc(ni_dataset.sentences, ni_dataset.labels, xl, xl_sampler))

acc_s, acc_nc, acc_ni

([0.05555555555555555,
  0.6666666666666666,
  0.9222222222222223,
  0.7777777777777778],
 [0.2111111111111111,
  0.4666666666666667,
  0.4888888888888889,
  0.6333333333333333],
 [0.29213483146067415,
  0.5730337078651685,
  0.6629213483146067,
  0.7415730337078652])

# Logit Difference Test

In [42]:
def get_batched_logit_diff(minibatch_size, tokens, answer_tokens, model):
    avg_logit_diff = 0

    for i in range(0, len(tokens), minibatch_size):

        target_index = i
        logit, _ = model.run_with_cache(tokens[target_index: target_index+minibatch_size])
        at = answer_tokens[target_index: target_index+minibatch_size]
        #print(logit.shape, answer_tokens.shape)
        logit_diff = logits_to_ave_logit_diff(logit, at, per_prompt=False)

        avg_logit_diff += logit_diff
        del logit
        del logit_diff

        print(i, avg_logit_diff)

    return avg_logit_diff/(len(tokens)/minibatch_size)



In [43]:
s_prompts = s_dataset.sentences
s_labels = s_dataset.labels
s_second_labels = s_dataset.B

s_answers = list(zip(s_labels, s_second_labels))
s_answer_tokens = t.concat([
    small.to_tokens(names, prepend_bos=False).T for names in s_answers
])

table = Table("Prompt", "Correct", "Incorrect", title="Prompts & Answers:")

for p, a in zip(s_prompts, s_answers):
    table.add_row(p, repr(a[0]), repr(a[1]))

In [44]:

#n_second_labels = [n_dataset.B[i] for i in c_idx_natural]
# for label, prompt in zip(labels, prompts):
#     logits, cache = model.run_with_cache(prompt, device=device, prepend_bos=False)
#     output = model.tokenizer.decode(logits[:,-1].argmax(dim=-1))
#     if output == label: # get highest wrong answer
#         output = model.tokenizer.decode((logits[:, -1].argsort(dim=-1))[0][-2]) # second maximum

#     #output = model.tokenizer.decode((logits[:, -1].argsort(dim=-1))[0][-2]) # always second maximum
#     second_labels.append(output)



In [45]:
#get the second maximum one
#s_second_labels = [s_dataset.B[i] for i in c_idx_symbolic]
ni_prompts = ni_dataset.sentences
ni_labels = ni_dataset.labels
ni_second_labels = ni_dataset.B


ni_answers = list(zip(ni_labels, ni_second_labels))
ni_answer_tokens = t.concat([
    small.to_tokens(names, prepend_bos=False).T for names in ni_answers
])


table = Table("Prompt", "Correct", "Incorrect", title="Prompts & Answers:")
for p, a in zip(ni_prompts, ni_answers):
    table.add_row(p, repr(a[0]), repr(a[1]))

rprint(table)



                                                Prompts & Answers:                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Prompt                                                                     ┃ Correct           ┃ Incorrect      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ All patients are cases. All cases are arguments. Therefore, all patients   │ ' arguments'      │ ' cases'       │
│ are                                                                        │                   │                │
│ All allegations are claims. All claims are rights. Therefore, all          │ ' rights'         │ ' claims'      │
│ allegations are                                                            │                   │                │
│ All wards are people. All people are judges. Therefore, all wards are      │ ' judges'         │ ' people'      │
│ All losers are people. All people are racists. Therefore, all losers are   │ ' racists'        │ ' people'      │
│ All aliens are people. All people are parents. Therefore, all aliens are   │ ' parents'        │ ' people'      │
│ All expectations are feelings. All feelings are sensitivity. Therefore,    │ ' sensitivity'    │ ' feelings'    │
│ all expectations are                                                       │                   │                │
│ All rituals are customs. All customs are trade. Therefore, all rituals are │ ' trade'          │ ' customs'     │
│ All pads are blocks. All blocks are housing. Therefore, all pads are       │ ' housing'        │ ' blocks'      │
│ All warehouses are houses. All houses are play. Therefore, all warehouses  │ ' play'           │ ' houses'      │
│ are                                                                        │                   │                │
│ All supervisors are people. All people are bullies. Therefore, all         │ ' bullies'        │ ' people'      │
│ supervisors are                                                            │                   │                │
│ All idiots are people. All people are blond. Therefore, all idiots are     │ ' blond'          │ ' people'      │
│ All farmers are people. All people are celebrities. Therefore, all farmers │ ' celebrities'    │ ' people'      │
│ are                                                                        │                   │                │
│ All plants are books. All books are sections. Therefore, all plants are    │ ' sections'       │ ' books'       │
│ All judges are people. All people are individuals. Therefore, all judges   │ ' individuals'    │ ' people'      │
│ are                                                                        │                   │                │
│ All subjects are people. All people are idiots. Therefore, all subjects    │ ' idiots'         │ ' people'      │
│ are                                                                        │                   │                │
│ All beds are surfaces. All surfaces are artifacts. Therefore, all beds are │ ' artifacts'      │ ' surfaces'    │
│ All wings are magazines. All magazines are press. Therefore, all wings are │ ' press'          │ ' magazines'   │
│ All learners are people. All people are scientists. Therefore, all         │ ' scientists'     │ ' people'      │
│ learners are                                                               │                   │                │
│ All bridges are structures. All structures are cognition. Therefore, all   │ ' cognition'      │ ' structures'  │
│ bridges are                                                                │                   │                │
│ All graduates are people. All people are chefs. Therefore, all graduates   │ ' chefs'          │ ' people'      │
│ are                                                   

In [46]:
nc_prompts = nc_dataset.sentences
nc_labels = nc_dataset.labels
nc_second_labels = nc_dataset.B

nc_answers = list(zip(nc_labels, nc_second_labels))
nc_answer_tokens = t.concat([
    small.to_tokens(names, prepend_bos=False).T for names in nc_answers
])


table = Table("Prompt", "Correct", "Incorrect", title="Prompts & Answers:")
for p, a in zip(nc_prompts, nc_answers):
    table.add_row(p, repr(a[0]), repr(a[1]))

rprint(table)


                                                Prompts & Answers:                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                                   ┃ Correct           ┃ Incorrect        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ All men are humans. All humans are mortal. Therefore, all men are        │ ' mortal'         │ ' humans'        │
│ All dens are units. All units are parts. Therefore, all dens are         │ ' parts'          │ ' units'         │
│ All pets are animals. All animals are organisms. Therefore, all pets are │ ' organisms'      │ ' animals'       │
│ All openings are starts. All starts are beginnings. Therefore, all       │ ' beginnings'     │ ' starts'        │
│ openings are                                                             │                   │                  │
│ All fronts are sides. All sides are surfaces. Therefore, all fronts are  │ ' surfaces'       │ ' sides'         │
│ All fences are boundaries. All boundaries are lines. Therefore, all      │ ' lines'          │ ' boundaries'    │
│ fences are                                                               │                   │                  │
│ All monitors are displays. All displays are demonstrations. Therefore,   │ ' demonstrations' │ ' displays'      │
│ all monitors are                                                         │                   │                  │
│ All necks are parts. All parts are objects. Therefore, all necks are     │ ' objects'        │ ' parts'         │
│ All governors are executives. All executives are administrators.         │ ' administrators' │ ' executives'    │
│ Therefore, all governors are                                             │                   │                  │
│ All explosions are increases. All increases are amounts. Therefore, all  │ ' amounts'        │ ' increases'     │
│ explosions are                                                           │                   │                  │
│ All compositions are creations. All creations are artifacts. Therefore,  │ ' artifacts'      │ ' creations'     │
│ all compositions are                                                     │                   │                  │
│ All taxis are cars. All cars are automobiles. Therefore, all taxis are   │ ' automobiles'    │ ' cars'          │
│ All books are objects. All objects are content. Therefore, all books are │ ' content'        │ ' objects'       │
│ All friendships are relationships. All relationships are relations.      │ ' relations'      │ ' relationships' │
│ Therefore, all friendships are                                           │                   │                  │
│ All pipes are tubes. All tubes are structures. Therefore, all pipes are  │ ' structures'     │ ' tubes'         │
│ All optics are properties. All properties are possessions. Therefore,    │ ' possessions'    │ ' properties'    │
│ all optics are                                                           │                   │                  │
│ All waves are gestures. All gestures are motion. Therefore, all waves    │ ' motion'         │ ' gestures'      │
│ are                                                                      │                   │                  │
│ All vectors are variables. All variables are quantities. Therefore, all  │ ' quantities'     │ ' variables'     │
│ vectors are                                                              │                   │                  │
│ All terminals are stations. All stations are facilities. Therefore, all  │ ' facilities'     │ ' stations'      │
│ terminals are                                                            │                   │                  │
│ All subtitles are titles. All titles are writing. Ther

### Small

In [47]:
s_tokens_small = small.to_tokens(s_prompts, prepend_bos=False).to(device)
ld_s = [get_batched_logit_diff(5, s_tokens_small, s_answer_tokens, small)]
print(ld_s)

0 tensor(-0.7883, device='cuda:0')
5 tensor(-1.7878, device='cuda:0')
10 tensor(-2.8699, device='cuda:0')
15 tensor(-3.9273, device='cuda:0')
20 tensor(-4.7970, device='cuda:0')
25 tensor(-5.5385, device='cuda:0')
30 tensor(-6.8633, device='cuda:0')
35 tensor(-7.6144, device='cuda:0')
40 tensor(-8.4015, device='cuda:0')
45 tensor(-9.1486, device='cuda:0')
50 tensor(-9.9973, device='cuda:0')
55 tensor(-10.9423, device='cuda:0')
60 tensor(-11.7716, device='cuda:0')
65 tensor(-12.6055, device='cuda:0')
70 tensor(-13.4525, device='cuda:0')
75 tensor(-14.3380, device='cuda:0')
80 tensor(-16.0682, device='cuda:0')
85 tensor(-16.6225, device='cuda:0')
[tensor(-0.9235, device='cuda:0')]


In [48]:
ni_tokens_small = small.to_tokens(ni_prompts, prepend_bos=False).to(device)
ld_ni = [get_batched_logit_diff(5, ni_tokens_small, ni_answer_tokens, small)]
print(ld_ni)

0 tensor(-0.8122, device='cuda:0')
5 tensor(-2.1591, device='cuda:0')
10 tensor(-2.2609, device='cuda:0')
15 tensor(-2.2328, device='cuda:0')
20 tensor(-3.4292, device='cuda:0')
25 tensor(-3.5476, device='cuda:0')
30 tensor(-5.0293, device='cuda:0')
35 tensor(-6.1920, device='cuda:0')
40 tensor(-6.8830, device='cuda:0')
45 tensor(-6.5963, device='cuda:0')
50 tensor(-6.8703, device='cuda:0')
55 tensor(-7.1306, device='cuda:0')
60 tensor(-7.6343, device='cuda:0')
65 tensor(-9.2314, device='cuda:0')
70 tensor(-11.6085, device='cuda:0')
75 tensor(-11.7739, device='cuda:0')
80 tensor(-12.2879, device='cuda:0')
85 tensor(-12.7332, device='cuda:0')
[tensor(-0.7153, device='cuda:0')]


In [49]:
nc_tokens_small = small.to_tokens(nc_prompts, prepend_bos=False).to(device)
ld_nc = [get_batched_logit_diff(5, nc_tokens_small, nc_answer_tokens, small)]
print(ld_nc)

0 tensor(-0.1091, device='cuda:0')
5 tensor(-0.8393, device='cuda:0')
10 tensor(-2.3075, device='cuda:0')
15 tensor(-4.0291, device='cuda:0')
20 tensor(-4.5164, device='cuda:0')
25 tensor(-5.1591, device='cuda:0')
30 tensor(-6.6525, device='cuda:0')
35 tensor(-7.3368, device='cuda:0')
40 tensor(-8.8919, device='cuda:0')
45 tensor(-10.3040, device='cuda:0')
50 tensor(-11.2563, device='cuda:0')
55 tensor(-12.1122, device='cuda:0')
60 tensor(-12.8542, device='cuda:0')
65 tensor(-14.1743, device='cuda:0')
70 tensor(-15.8297, device='cuda:0')
75 tensor(-17.3244, device='cuda:0')
80 tensor(-17.5054, device='cuda:0')
85 tensor(-19.7828, device='cuda:0')
[tensor(-1.0990, device='cuda:0')]


### Medium

In [50]:
s_tokens_medium = medium.to_tokens(s_prompts, prepend_bos=False).to(device)
ld_s.append(get_batched_logit_diff(5, s_tokens_medium, s_answer_tokens, medium))
print(ld_s)

0 tensor(0.6036, device='cuda:0')
5 tensor(0.9981, device='cuda:0')
10 tensor(1.7193, device='cuda:0')
15 tensor(2.3201, device='cuda:0')
20 tensor(2.6465, device='cuda:0')
25 tensor(3.1680, device='cuda:0')
30 tensor(3.4318, device='cuda:0')
35 tensor(4.0275, device='cuda:0')
40 tensor(4.2168, device='cuda:0')
45 tensor(5.0610, device='cuda:0')
50 tensor(5.4465, device='cuda:0')
55 tensor(5.7920, device='cuda:0')
60 tensor(6.1986, device='cuda:0')
65 tensor(6.6282, device='cuda:0')
70 tensor(6.7765, device='cuda:0')
75 tensor(6.8240, device='cuda:0')
80 tensor(6.3177, device='cuda:0')
85 tensor(6.6440, device='cuda:0')
[tensor(-0.9235, device='cuda:0'), tensor(0.3691, device='cuda:0')]


In [51]:
ni_tokens_medium = medium.to_tokens(ni_prompts, prepend_bos=False).to(device)
ld_ni.append(get_batched_logit_diff(5, ni_tokens_medium, ni_answer_tokens, medium))
print(ld_ni)

0 tensor(0.5361, device='cuda:0')
5 tensor(0.5937, device='cuda:0')
10 tensor(1.6581, device='cuda:0')
15 tensor(3.2934, device='cuda:0')
20 tensor(3.7193, device='cuda:0')
25 tensor(3.7512, device='cuda:0')
30 tensor(4.7835, device='cuda:0')
35 tensor(5.1096, device='cuda:0')
40 tensor(5.9461, device='cuda:0')
45 tensor(8.0429, device='cuda:0')
50 tensor(8.2734, device='cuda:0')
55 tensor(10.6593, device='cuda:0')
60 tensor(11.5638, device='cuda:0')
65 tensor(10.7986, device='cuda:0')
70 tensor(9.8279, device='cuda:0')
75 tensor(9.0347, device='cuda:0')
80 tensor(9.7786, device='cuda:0')
85 tensor(10.8153, device='cuda:0')
[tensor(-0.7153, device='cuda:0'), tensor(0.6076, device='cuda:0')]


In [52]:
nc_tokens_medium = medium.to_tokens(nc_prompts, prepend_bos=False).to(device)
ld_nc.append(get_batched_logit_diff(5, nc_tokens_medium, nc_answer_tokens, medium))
print(ld_nc)

0 tensor(0.9112, device='cuda:0')
5 tensor(1.5885, device='cuda:0')
10 tensor(1.0376, device='cuda:0')
15 tensor(1.9764, device='cuda:0')
20 tensor(3.7019, device='cuda:0')
25 tensor(3.2372, device='cuda:0')
30 tensor(3.0283, device='cuda:0')
35 tensor(3.4555, device='cuda:0')
40 tensor(4.0896, device='cuda:0')
45 tensor(3.5588, device='cuda:0')
50 tensor(3.8360, device='cuda:0')
55 tensor(4.4316, device='cuda:0')
60 tensor(4.4420, device='cuda:0')
65 tensor(4.8359, device='cuda:0')
70 tensor(4.2089, device='cuda:0')
75 tensor(4.3720, device='cuda:0')
80 tensor(4.3163, device='cuda:0')
85 tensor(4.7399, device='cuda:0')
[tensor(-1.0990, device='cuda:0'), tensor(0.2633, device='cuda:0')]


### Large

In [53]:
s_tokens_large = large.to_tokens(s_prompts, prepend_bos=False).to(device)
ld_s.append(get_batched_logit_diff(5, s_tokens_large, s_answer_tokens, large))
print(ld_s)

0 tensor(1.0546, device='cuda:0')
5 tensor(2.1778, device='cuda:0')
10 tensor(3.2005, device='cuda:0')
15 tensor(4.2704, device='cuda:0')
20 tensor(5.5789, device='cuda:0')
25 tensor(6.1238, device='cuda:0')
30 tensor(6.9293, device='cuda:0')
35 tensor(7.7327, device='cuda:0')
40 tensor(8.8898, device='cuda:0')
45 tensor(10.1495, device='cuda:0')
50 tensor(10.9977, device='cuda:0')
55 tensor(11.7997, device='cuda:0')
60 tensor(12.4906, device='cuda:0')
65 tensor(14.2091, device='cuda:0')
70 tensor(15.1846, device='cuda:0')
75 tensor(16.2604, device='cuda:0')
80 tensor(16.1204, device='cuda:0')
85 tensor(17.0625, device='cuda:0')
[tensor(-0.9235, device='cuda:0'), tensor(0.3691, device='cuda:0'), tensor(0.9479, device='cuda:0')]


In [54]:
ni_tokens_large = large.to_tokens(ni_prompts, prepend_bos=False).to(device)
ld_ni.append(get_batched_logit_diff(5, ni_tokens_large, ni_answer_tokens, large))
print(ld_ni)

0 tensor(0.7417, device='cuda:0')
5 tensor(0.7775, device='cuda:0')
10 tensor(1.7604, device='cuda:0')
15 tensor(3.6522, device='cuda:0')
20 tensor(4.0335, device='cuda:0')
25 tensor(4.5271, device='cuda:0')
30 tensor(5.6808, device='cuda:0')
35 tensor(6.7603, device='cuda:0')
40 tensor(7.4182, device='cuda:0')
45 tensor(9.9754, device='cuda:0')
50 tensor(10.8468, device='cuda:0')
55 tensor(12.0309, device='cuda:0')
60 tensor(11.6353, device='cuda:0')
65 tensor(10.2145, device='cuda:0')
70 tensor(10.0056, device='cuda:0')
75 tensor(9.4727, device='cuda:0')
80 tensor(10.6291, device='cuda:0')
85 tensor(11.9739, device='cuda:0')
[tensor(-0.7153, device='cuda:0'), tensor(0.6076, device='cuda:0'), tensor(0.6727, device='cuda:0')]


In [55]:
nc_tokens_large = large.to_tokens(nc_prompts, prepend_bos=False).to(device)
ld_nc.append(get_batched_logit_diff(5, nc_tokens_large, nc_answer_tokens, large))
print(ld_nc)

0 tensor(1.3205, device='cuda:0')
5 tensor(2.6575, device='cuda:0')
10 tensor(2.7648, device='cuda:0')
15 tensor(3.1986, device='cuda:0')
20 tensor(4.6015, device='cuda:0')
25 tensor(4.3589, device='cuda:0')
30 tensor(4.3616, device='cuda:0')
35 tensor(5.7523, device='cuda:0')
40 tensor(6.2349, device='cuda:0')
45 tensor(6.0670, device='cuda:0')
50 tensor(5.8958, device='cuda:0')
55 tensor(5.0099, device='cuda:0')
60 tensor(5.2260, device='cuda:0')
65 tensor(5.8421, device='cuda:0')
70 tensor(5.4165, device='cuda:0')
75 tensor(5.2501, device='cuda:0')
80 tensor(4.3978, device='cuda:0')
85 tensor(4.6918, device='cuda:0')
[tensor(-1.0990, device='cuda:0'), tensor(0.2633, device='cuda:0'), tensor(0.2607, device='cuda:0')]


### XL

In [56]:
s_tokens_xl = xl.to_tokens(s_prompts, prepend_bos=False).to(device)
ld_s.append(get_batched_logit_diff(5, s_tokens_xl, s_answer_tokens, xl))
print(ld_s)

0 tensor(0.6644, device='cuda:0')
5 tensor(2.0852, device='cuda:0')
10 tensor(2.6657, device='cuda:0')
15 tensor(3.4140, device='cuda:0')
20 tensor(4.7138, device='cuda:0')
25 tensor(4.4549, device='cuda:0')
30 tensor(5.6515, device='cuda:0')
35 tensor(6.0557, device='cuda:0')
40 tensor(6.9096, device='cuda:0')
45 tensor(7.7677, device='cuda:0')
50 tensor(8.2122, device='cuda:0')
55 tensor(10.0149, device='cuda:0')
60 tensor(11.1347, device='cuda:0')
65 tensor(11.6900, device='cuda:0')
70 tensor(12.3471, device='cuda:0')
75 tensor(13.5938, device='cuda:0')
80 tensor(13.9224, device='cuda:0')
85 tensor(14.8966, device='cuda:0')
[tensor(-0.9235, device='cuda:0'), tensor(0.3691, device='cuda:0'), tensor(0.9479, device='cuda:0'), tensor(0.8276, device='cuda:0')]


In [57]:
ni_tokens_xl = xl.to_tokens(ni_prompts, prepend_bos=False).to(device)
ld_ni.append(get_batched_logit_diff(5, ni_tokens_xl, ni_answer_tokens, xl))
print(ld_ni)

0 tensor(1.1883, device='cuda:0')
5 tensor(1.3769, device='cuda:0')
10 tensor(3.6169, device='cuda:0')
15 tensor(6.5605, device='cuda:0')
20 tensor(8.0324, device='cuda:0')
25 tensor(10.0055, device='cuda:0')
30 tensor(11.7642, device='cuda:0')
35 tensor(13.4073, device='cuda:0')
40 tensor(14.6950, device='cuda:0')
45 tensor(16.5292, device='cuda:0')
50 tensor(17.6696, device='cuda:0')
55 tensor(19.6915, device='cuda:0')
60 tensor(19.8011, device='cuda:0')
65 tensor(19.3359, device='cuda:0')
70 tensor(19.8507, device='cuda:0')
75 tensor(19.3170, device='cuda:0')
80 tensor(21.7346, device='cuda:0')
85 tensor(23.6638, device='cuda:0')
[tensor(-0.7153, device='cuda:0'), tensor(0.6076, device='cuda:0'), tensor(0.6727, device='cuda:0'), tensor(1.3294, device='cuda:0')]


In [58]:
nc_tokens_xl = xl.to_tokens(nc_prompts, prepend_bos=False).to(device)
ld_nc.append(get_batched_logit_diff(5, nc_tokens_xl, nc_answer_tokens, xl))
print(ld_nc)

0 tensor(2.3001, device='cuda:0')
5 tensor(3.6619, device='cuda:0')
10 tensor(4.9248, device='cuda:0')
15 tensor(6.0585, device='cuda:0')
20 tensor(8.3055, device='cuda:0')
25 tensor(8.5004, device='cuda:0')
30 tensor(9.8705, device='cuda:0')
35 tensor(12.5430, device='cuda:0')
40 tensor(13.2190, device='cuda:0')
45 tensor(13.8041, device='cuda:0')
50 tensor(15.5406, device='cuda:0')
55 tensor(16.8898, device='cuda:0')
60 tensor(17.3966, device='cuda:0')
65 tensor(18.0317, device='cuda:0')
70 tensor(18.3101, device='cuda:0')
75 tensor(19.3676, device='cuda:0')
80 tensor(19.8072, device='cuda:0')
85 tensor(20.5547, device='cuda:0')
[tensor(-1.0990, device='cuda:0'), tensor(0.2633, device='cuda:0'), tensor(0.2607, device='cuda:0'), tensor(1.1419, device='cuda:0')]


# Plot the Accuracy and LD

In [59]:
# seed 42
acc_s = [a * 100 for a in acc_s]
acc_ni = [a * 100 for a in acc_ni]
acc_nc = [a * 100 for a in acc_nc]

In [60]:
sequence_labels = ['Small (117M)', 'Medium (345M)', 'Large (762M)', 'XL (1.5B)']

accuracy

In [93]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

# Assuming the variables are defined
y1 = acc_s
y2 = acc_nc
y3 = acc_ni
labels = sequence_labels

# Compute average across categories (labels)
average_line = np.mean([y1, y2, y3], axis=0)

# Create bar plots for each dataset
fig = go.Figure()
fig.add_trace(go.Bar(x=labels, y=y1, name='Symbolic', marker_color='#1f77b4'))  # Blue
fig.add_trace(go.Bar(x=labels, y=y2, name='Belief Consistent', marker_color='#ff7f0e'))  # Orange
# fig.add_trace(go.Bar(x=labels, y=y3, name='Belief Inconsistent', marker_color='#2ca02c'))  # Green

# Add average line as a scatter plot
fig.add_trace(go.Scatter(x=labels, y=average_line, mode='lines+markers', name='Average',
                         line=dict(color='black', width=1, dash='dash'),
                         marker=dict(symbol='diamond-open', size=5, color='black', line=dict(color='black', width=1))))

# Update layout with titles and axis labels
fig.update_layout(
    # title='Model Selection Accuracy',
    xaxis_title='Model Size',
    yaxis_title='Accuracy (%)',
    legend=dict(
        x=0.25,
        y=0.97,
        xanchor='center',
        yanchor='top',
        orientation='h',
        font=dict(size=13),
        bgcolor='rgba(255,255,255,0.7)',
        # bordercolor='black',
        # borderwidth=1
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Adjust margins for better spacing
    plot_bgcolor='white',  # White background
    barmode='group',  # Group bars for each label
    bargap=0.2,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    width=800,
    height=500,
    xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, title_font=dict(size=20)),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, title_font=dict(size=20)),
)

# Show figure
fig.show()

# Save figure
pio.write_image(fig, './FIGURE/model_selection_1.png', scale=3)  # Adjust width for publication quality


In [63]:
ld_s = [i.cpu() for i in ld_s]
ld_nc = [i.cpu() for i in ld_nc]
ld_ni = [i.cpu() for i in ld_ni]

In [94]:

# Assuming the variables are defined
y1 = ld_s
y2 = ld_nc
y3 = ld_ni
labels = sequence_labels

# Compute average across categories (labels)
average_line = np.mean([y1, y2, y3], axis=0)

# Create bar plots for each dataset
fig = go.Figure()
fig.add_trace(go.Bar(x=labels, y=y1, name='Symbolic', marker_color='#1f77b4'))  # Blue
fig.add_trace(go.Bar(x=labels, y=y2, name='Belief Consistent', marker_color='#ff7f0e'))  # Orange
#fig.add_trace(go.Bar(x=labels, y=y3, name='Belief Inconsistent', marker_color='#2ca02c'))  # Green

# Add average line as a scatter plot
fig.add_trace(go.Scatter(x=labels, y=average_line, mode='lines+markers', name='Average',
                         line=dict(color='black', width=1, dash='dash'),
                         marker=dict(symbol='diamond-open', size=5, color='black', line=dict(color='black', width=1))))

# Update layout with titles and axis labels
fig.update_layout(
    # title='Model Selection Accuracy',
    xaxis_title='Model Size',
    yaxis_title='Accuracy (%)',
    legend=dict(
        x=0.25,
        y=0.97,
        xanchor='center',
        yanchor='top',
        orientation='h',
        font=dict(size=13),
        bgcolor='rgba(255,255,255,0.7)',
        # bordercolor='black',
        # borderwidth=1
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Adjust margins for better spacing
    plot_bgcolor='white',  # White background
    barmode='group',  # Group bars for each label
    bargap=0.2,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    width=800,
    height=500,
    xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, title_font=dict(size=20)),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, title_font=dict(size=20)),
)
# Show figure
fig.show()

# Save figure
pio.write_image(fig, './FIGURE/model_selection_2.png', scale=3)  # Adjust width for publication quality


In [ ]:
# Assuming the variables are defined
y1_1 = acc_s
y2_1 = acc_nc
y3_1 = acc_ni
y1_2 = ld_s
y2_2 = ld_nc
y3_2 = ld_ni
labels = sequence_labels

# Compute averages across categories (labels) for both datasets
average_line_1 = np.mean([y1_1, y2_1, y3_1], axis=0)
average_line_2 = np.mean([y1_2, y2_2, y3_2], axis=0)

# Create subplots with shared legend
fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Logit Difference'))

# Add bar plots and average lines for Accuracy subplot
fig.add_trace(go.Bar(x=labels, y=y1_1, name='Symbolic', marker_color='#1f77b4'), row=1, col=1)
fig.add_trace(go.Bar(x=labels, y=y2_1, name='Belief Consistent', marker_color='#ff7f0e'), row=1, col=1)
fig.add_trace(go.Bar(x=labels, y=y3_1, name='Belief Inconsistent', marker_color='#2ca02c'), row=1, col=1)
fig.add_trace(go.Scatter(x=labels, y=average_line_1, mode='lines+markers', name='Average',
                         line=dict(color='black', width=1, dash='dash'),
                         marker=dict(symbol='diamond-open', size=5, color='black', line=dict(color='black', width=1))),
                         row=1, col=1)

# Add bar plots and average lines for Logit Difference subplot
fig.add_trace(go.Bar(x=labels, y=y1_2, name='Symbolic', marker_color='#1f77b4', showlegend=False), row=1, col=2)
fig.add_trace(go.Bar(x=labels, y=y2_2, name='Belief Consistent', marker_color='#ff7f0e', showlegend=False), row=1, col=2)
fig.add_trace(go.Bar(x=labels, y=y3_2, name='Belief Inconsistent', marker_color='#2ca02c', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=labels, y=average_line_2, mode='lines+markers', name='Average', showlegend=False,
                         line=dict(color='black', width=1, dash='dash'),
                         marker=dict(symbol='diamond-open', size=5, color='black', line=dict(color='black', width=1))),
                         row=1, col=2)

# Update layout with titles and axis labels
fig.update_layout(
    xaxis_title='GPT-2 Model Size',
    xaxis2_title = 'GPT-2 Model Size',
    yaxis_title='Accuracy (%)',
    yaxis2_title='Logit Difference',
    plot_bgcolor='white',  # White background
    bargap=0.2,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    # Add borders
    xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True),
)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, row=1, col=2)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, row=1, col=2)

# Update legends and margins
fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.3,
        xanchor='center',
        yanchor='top',
        orientation='h',
    ),
    margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins for better spacing
)

# Show figure
fig.show()

# Save figure
pio.write_image(fig, './FIGURE/model_selection_1.png', scale=3, width=1000)  # Adjust width for publication quality


# Top-k plot

In [ ]:
def get_rank(logits, answer_tokens):
    batch_final_logits = logits[:, -1, :]
    batch_ranks = []
    for i, final_logits in enumerate(batch_final_logits):
        final_logits_list = final_logits.tolist()
        value_at_index = final_logits_list[answer_tokens[i][0]]
        sorted_final_logits_list = sorted(final_logits_list, reverse=True)
        rank_of_value = sorted_final_logits_list.index(value_at_index) + 1
        batch_ranks.append(rank_of_value)

    return batch_ranks

In [ ]:
s_logits, _ = medium.run_with_cache(s_prompts, device=device, prepend_bos=False)
s_rank = get_rank(s_logits, s_answer_tokens)
s_hist, s_hist_label = np.histogram(s_rank, bins=range(1, 21))
s_hist

array([60, 28,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [ ]:
ni_logits, _ = medium.run_with_cache(ni_prompts, device=device, prepend_bos=False)

In [ ]:
ni_rank = get_rank(ni_logits, ni_answer_tokens)
ni_hist, ni_hist_label = np.histogram(ni_rank, bins=range(1, 21))
ni_hist

array([51, 21,  7,  0,  4,  1,  2,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0])

In [ ]:
nc_logits, _ = medium.run_with_cache(nc_prompts, device=device, prepend_bos=False)
nc_rank = get_rank(nc_logits, nc_answer_tokens)
nc_hist, nc_hist_label = np.histogram(nc_rank, bins=range(1, 21))

In [ ]:
nc_hist

array([41, 17, 13,  6,  4,  1,  0,  3,  0,  0,  2,  0,  0,  0,  0,  0,  0,
        0,  0])

In [ ]:
# # seed 317
# s_hist = [323, 151,   6,   0,   0,   0,   0,   0,   0]
# ni_hist = [257, 101,  43,  22,  13,   3,   6,   2,   3]
# nc_hist = [266,  85,  37,  14,  14,   7,   4,   7,   4]

In [ ]:
label = [0] + nc_hist_label

In [ ]:
label = np.arange(0, 10)
label

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
import plotly.graph_objects as go

# Assuming the variables are defined
y1 = s_hist  # Symbolic
y2 = nc_hist  # Belief Consistent
y3 = ni_hist  # Belief Inconsistent
labels = label  # Labels for each point on the x-axis

# Create line plots for each dataset with filled areas and smooth lines
fig = go.Figure()
fig.add_trace(go.Scatter(x=labels, y=y1, mode='lines', name='Symbolic',
                         line=dict(color='#1f77b4'),  line_shape='spline'))  # Blue
fig.add_trace(go.Scatter(x=labels, y=y2, mode='lines', name='Belief Consistent',
                         line=dict(color='#ff7f0e'),line_shape='spline'))  # Orange
fig.add_trace(go.Scatter(x=labels, y=y3, mode='lines', name='Belief Inconsistent',
                         line=dict(color='#2ca02c'),  line_shape='spline'))  # Green

# Update layout with titles and axis labels
fig.update_layout(
    xaxis_title='Model Size',
    yaxis_title='Accuracy (%)',
    legend=dict(
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        orientation='h',
        font=dict(size=12)
    ),
    margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins for better spacing
    plot_bgcolor='white',  # White background
    xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True)
)

# Show the figure
fig.show()

# Optionally save the figure if running in an environment that supports it
# pio.write_image(fig, 'rank_prediction_smooth.png', scale=3)  # Adjust width for publication quality


In [ ]:
import plotly.graph_objects as go

# Assuming the variables are defined
y1 = s_hist  # Symbolic
y2 = nc_hist  # Belief Consistent
y3 = ni_hist  # Belief Inconsistent
labels = nc_hist_label # Labels for each point on the x-axis

# Create bar plots for each dataset
fig = go.Figure()
fig.add_trace(go.Bar(x=labels, y=y1, name='Symbolic', marker_color='#1f77b4'))  # Blue
fig.add_trace(go.Bar(x=labels, y=y2, name='Belief Consistent', marker_color='#ff7f0e'))  # Orange
fig.add_trace(go.Bar(x=labels, y=y3, name='Belief Inconsistent', marker_color='#2ca02c'))  # Green

# Update layout with titles and axis labels
fig.update_layout(
    xaxis_title='Prediction Ranks for the Correct Token',
    yaxis_title='Number of Samples',
    legend=dict(
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        orientation='h',
        font=dict(size=12)
    ),
    margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins for better spacing
    plot_bgcolor='white',  # White background
    barmode='group',  # Group bars for each label to enable comparison
    bargap=0.15,  # Gap between bars within the same group
    bargroupgap=0.1,  # Gap between groups
    xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True)
)

# Show the figure
fig.show()

# Optionally save the figure if running in an environment that supports it
pio.write_image(fig, './FIGURE/model_selection_2.png', scale=3, width=500)  # Adjust width for publication quality
